# Load and Explore customers data from bronze layer and load to silver layer

This section loads the `customers_silver` table from the silver layer in Azure Databricks. The table contains customer details such as `customer_id`, `first_name`, `last_name`, `city`, and `state`. You can use this data for further analysis or joining with other tables.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@azuresadatalake.dfs.core.windows.net/customers")

In [0]:
df.display()

In [0]:
df.describe()

In [0]:
df = df.drop("_rescued_data")

df.show(10)

In [0]:
df.printSchema()

In [0]:
df = df.withColumn("email_domain", split(col("email"), "@")[1])
df.display()


In [0]:
df.groupBy("email_domain").agg(count("customer_id").alias("total_customers")).sort("total_customers", ascending=False).display()

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))
df.display()


In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@azuresadatalake.dfs.core.windows.net/customers")

In [0]:
%sql

create schema if not exists databricks_catalog.silver;

In [0]:
%sql

create table if not exists databricks_catalog.silver.customers_silver using delta 
location "abfss://silver@azuresadatalake.dfs.core.windows.net/customers"